In [ ]:
# !pip install -e ".[dev]"

In [ ]:
# %load_ext autoreload
# %autoreload 2
# %env ANYWIDGET_HMR=1

import numpy as np
import pandas as pd

from dimbridge import Dimbridge

In [ ]:
x = np.random.rand(10000) * 5
y = np.random.rand(10000) * 2
z = np.cos(x + y)
w = x**2

df = pd.DataFrame(dict(d10=x, d2=y, d3=z, d4=w))

In [ ]:
# from umap import UMAP
# xy = UMAP(n_neighbors=50).fit_transform(np.c_[x, y, z, w])
xy = np.c_[x, y]

In [ ]:
# Testing
from importlib import reload

import dimbridge

reload(dimbridge)
from dimbridge import Dimbridge

dimbridge = Dimbridge(
    data=df,
    x=xy[:, 0],
    y=xy[:, 1],
    c=np.repeat([[255,0,0]], xy.shape[0], 0),
    s=np.zeros_like(x) + 4,  # mark size
)
dimbridge

In [ ]:
dimbridge.selected

In [ ]:
# todo export svg / pdf
# todo how-to, readme
#